# Extract an Intermediate Layer

## Method 1: Registering a forward hook on a certain layer of the network

* define one `get_activation()` function

    ``` python
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    ```

* register_forward_hook on the layer you want to extract

    ``` python
    model.fc2.register_forward_hook(get_activation('fc2'))
    ```

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.cl1 = nn.Linear(25, 60)
        self.cl2 = nn.Linear(60, 16)
        self.fc1 = nn.Linear(16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.relu(self.cl1(x))
        x = F.relu(self.cl2(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x


activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


model = MyModel()
model.fc2.register_forward_hook(get_activation('fc2'))
x = torch.randn(1, 25)
output = model(x)
print(activation['fc2'])

tensor([[-0.0114, -0.0545, -0.1958, -0.0217,  0.0052, -0.0270, -0.1623, -0.0467,
          0.0420,  0.0792, -0.0398, -0.1767, -0.0108, -0.0066,  0.0003, -0.0294,
          0.1365,  0.2850,  0.0139,  0.0335,  0.0147,  0.0164, -0.1176,  0.0268,
          0.0120,  0.0711,  0.0201,  0.0755, -0.0263,  0.1325,  0.0510,  0.0175,
         -0.0562, -0.0440, -0.0282,  0.2546,  0.0930,  0.2249,  0.0215, -0.0762,
         -0.2013,  0.0587,  0.0408, -0.1045,  0.0395, -0.0668, -0.0763,  0.0857,
          0.1199, -0.0237,  0.0630, -0.1114,  0.2050,  0.0433, -0.0524,  0.0951,
          0.0485,  0.1931,  0.0054, -0.1557, -0.1906, -0.1204, -0.0229, -0.1178,
         -0.0255, -0.1037,  0.0319, -0.0813,  0.0408, -0.0681, -0.0855, -0.1064,
          0.1218,  0.0064, -0.0844, -0.0134, -0.0517,  0.1180, -0.0070, -0.0217,
          0.0549, -0.0024,  0.0295, -0.0462]])


Extract all layers

In [6]:
for name, layer in model.named_modules():
    layer.register_forward_hook(get_activation(name))

x = torch.randn(1, 25)
output = model(x)
for key in activation:
    print(activation[key])

tensor([[ 0.0593, -0.0217, -0.1351, -0.0384,  0.0221, -0.0265, -0.1854, -0.0423,
         -0.0430,  0.0263, -0.0571, -0.1501,  0.0137,  0.0437,  0.0296, -0.0021,
          0.1003,  0.2045,  0.0583,  0.0202,  0.0149,  0.0095, -0.1051, -0.0425,
         -0.0082,  0.0107,  0.0240,  0.0963, -0.0122,  0.0945,  0.0904,  0.0995,
         -0.0547, -0.0915, -0.0320,  0.2111,  0.1059,  0.2046,  0.0776, -0.0775,
         -0.1315,  0.0776, -0.0063, -0.0944,  0.0301, -0.0324, -0.0726,  0.0483,
          0.0622, -0.0694,  0.1037, -0.1082,  0.1846,  0.0397, -0.0064,  0.0464,
          0.0380,  0.1812, -0.0057, -0.1008, -0.1975, -0.1179, -0.0587, -0.0972,
         -0.0103, -0.0964,  0.0199, -0.1028, -0.0028,  0.0164, -0.0622, -0.0736,
          0.0811, -0.0414, -0.0015, -0.1005, -0.0189,  0.0445, -0.0213,  0.0140,
          0.0217,  0.0507,  0.0098, -0.0195]])
tensor([[ 0.0728, -0.5055,  0.0482, -0.8321,  0.0047, -0.3677,  0.2545, -0.4243,
          0.8094, -0.7969, -0.3151, -0.3689,  0.4332, -0.1288,

## Method 2: Use torch_intermediate_layer_getter

### Install torch_intermediate_layer_getter

In [7]:
pip install torch_intermediate_layer_getter

Processing /home/hongbing/.cache/pip/wheels/7d/f4/b2/0793f47b888179b3a89e3d6e0c53180903e5cd3a6ea1ec4b95/torch_intermediate_layer_getter-0.1.post1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_intermediate_layer_getter import IntermediateLayerGetter as MidGetter

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.cl1 = nn.Linear(25, 60)
        self.cl2 = nn.Linear(60, 16)
        self.fc1 = nn.Linear(16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.relu(self.cl1(x))
        x = F.relu(self.cl2(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x), dim=1)
        return x


model = MyModel()
return_layers = {
    'fc2': 'fc2',
}
mid_getter = MidGetter(model, return_layers, keep_output=True)
x = torch.randn(1, 25)
mid_outputs, model_output = mid_getter(x)
print(mid_outputs)

OrderedDict([('fc2', tensor([[ 4.5227e-02,  3.9497e-02, -5.3636e-02,  6.4118e-02,  8.0344e-02,
          1.2241e-01, -6.7592e-02,  1.3136e-01, -1.6434e-01,  9.6387e-02,
          1.1345e-03,  2.2225e-03,  1.2926e-01,  2.5593e-01,  9.0569e-02,
         -2.7969e-02,  1.7415e-01, -1.4138e-01, -1.9583e-01,  9.9759e-02,
          3.9906e-02, -1.4257e-02, -1.2426e-01, -1.4839e-01, -3.7735e-02,
         -4.1160e-02, -5.3619e-02, -1.1741e-02,  1.3697e-03, -1.9188e-02,
         -5.3952e-02, -2.6832e-02,  1.1674e-01,  4.0707e-02, -6.7977e-02,
         -2.2417e-02,  1.0721e-01,  1.2081e-01,  5.5723e-03,  7.8668e-02,
          1.0027e-02,  1.5414e-01,  5.1735e-02,  5.9609e-03, -6.2577e-02,
         -1.5737e-01,  1.2430e-01,  5.0084e-02,  3.8888e-02,  1.6673e-02,
          8.9586e-02, -4.3854e-02,  9.3739e-02, -6.8255e-02, -9.1607e-02,
          2.9572e-02,  1.9342e-02,  1.9910e-01,  2.9072e-02,  4.0594e-02,
         -3.3934e-02,  7.1060e-05, -2.2376e-02,  3.8505e-02,  1.0437e-02,
          1.4113e